In [2]:
import numpy as np
import pandas as pd
import sys

from datetime import date
from utils import *

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 100)
sys.path.insert(0, 'C:\\Users\\valen\\OneDrive\\Documenti\\GSQSA\\gsqsa_basket\\src')

In [3]:
data_path = "C:/Users/valen/OneDrive/Documenti/GSQSA/gsqsa_basket/input_data/"
output_path = "C:/Users/valen/OneDrive/Documenti/GSQSA/gsqsa_basket/output/"

# read all sheets into dict
df_dict = pd.read_excel(data_path+"tabellini.xlsx", sheet_name=None)

players_bio = pd.read_excel(data_path+"players_bio.xlsx")

In [4]:
if __name__ == "__main__":

    data_path = "C:/Users/valen/OneDrive/Documenti/GSQSA/gsqsa_basket/input_data/"
    output_path = "C:/Users/valen/OneDrive/Documenti/GSQSA/gsqsa_basket/output/"
    
    # read all sheets into dict
    df_dict = pd.read_excel(data_path+"tabellini.xlsx", sheet_name=None)

    # validity checks
    games_list, opp_FTM_list, opp_FTA_list, opp_2PM_list, opp_2PA_list, opp_3PM_list, opp_3PA_list = ([] for i in range(7))
    for tab in df_dict.values():

        # checks
        tab_check = tab[~tab.NUMERO.isin([98,99])]
        assert ((tab_check.FTM > tab_check.FTA).sum()==0),"Free Throws Made can't be greater than Free Throws Attempted"
        assert ((tab_check['2PM'] > tab_check['2PA']).sum()==0),"2 Points Made can't be greater than 2 Points Attempted"
        assert ((tab_check['3PM'] > tab_check['3PA']).sum()==0),"3 Points Made can't be greater than 3 Points Attempted"
        assert ((tab_check.FF > 5).sum()==0), "A player can't make more than 5 fouls"

        #get opponent team name (if 99 GSQSA played home, else played away)
        if tab.iloc[-1].NUMERO == 99:
            games_list.append("vs "+tab.iloc[-1].PLAYER)
        else:
            games_list.append("@ "+tab.iloc[-1].PLAYER)

        # opponent points
        opp_FTM_list.append(tab.iloc[-1].FTM)
        opp_FTA_list.append(tab.iloc[-1].FTA)
        opp_2PM_list.append(tab.iloc[-1]['2PM'])
        opp_2PA_list.append(tab.iloc[-1]['2PA'])
        opp_3PM_list.append(tab.iloc[-1]['3PM'])
        opp_3PA_list.append(tab.iloc[-1]['3PA'])
        
    # read players bio
    players_bio = pd.read_excel(data_path+"players_bio.xlsx")
    players_bio['Age'] = players_bio.BORN.apply(lambda x: age(x))
    
    # clean df
    df_dict_cleaned = {k:clean_stats(v) for k, v in df_dict.items()}

    # add game number and calculate teams stats
    for i, k in enumerate(df_dict_cleaned.keys()):
        
        # game number
        df_dict_cleaned[k]['game'] = i+1

        # team stats
        tmp = pd.DataFrame(df_dict_cleaned[k][team_cols].sum()).T
        tmp.rename(columns={0:i})
        if i == 0:
            team_stats = tmp
        else:
            team_stats = pd.concat([team_stats,tmp])
        team_stats = team_stats.reset_index().drop('index',axis=1).astype(int)

    team_stats = clean_team_stats(team_stats)
    team_stats['Game'] = games_list

    # opponent team stats
    opp_team_stats = pd.DataFrame({
        'Game':games_list,
        'FTM':opp_FTM_list,
        'FTA':opp_FTA_list,
        '2PM':opp_2PM_list,
        '2PA':opp_2PA_list,
        '3PM':opp_3PM_list,
        '3PA':opp_3PA_list
    })
    opp_team_stats['PTS'] = opp_team_stats['FTM'] + 2*opp_team_stats['2PM'] + 3*opp_team_stats['3PM']

    # concat tabs
    tabs = pd.concat(list(df_dict_cleaned.values()))

    # merge with players bio
    tabs = pd.merge(
        tabs,
        players_bio.drop(['NUMBER','BORN'],axis=1),
        on='PLAYER',
        how='left'
    )

    # aggregates
    tab_mean = tabs.groupby('PLAYER', as_index=False).agg({
        'Age':'max',
        'POS':'max',
        'HEIGHT':'max',
        'PTS':'mean',
        'FGM':'mean',
        'FGA':'mean',
        '3PM':'mean', 
        '3PA':'mean',
        '2PM':'mean',
        '2PA':'mean',
        'FTM':'mean',
        'FTA':'mean',       
        'RO':'mean',
        'RD':'mean',
        'RT':'mean',
        'AST':'mean',
        'PP':'mean',
        'PR':'mean',
        'ST':'mean',
        'FF':'mean',
        'FS':'mean',
        '+/-':'mean',
        'MIN':'mean'}).round(2)#.rename(columns={'size':'G'})

    tab_sum = tabs.groupby('PLAYER', as_index=False).agg({
        'Age':'max',
        'POS':'max',
        'HEIGHT':'max',
        'PTS':'sum',
        'FTM':'sum',
        'FTA':'sum',
        'FGM':'sum',
        'FGA':'sum',
        '2PM':'sum',
        '2PA':'sum',
        '3PM':'sum',
        '3PA':'sum',
        'RO':'sum',
        'RD':'sum',
        'RT':'sum',
        'AST':'sum',
        'PP':'sum',
        'PR':'sum',
        'ST':'sum',
        'FF':'sum',
        'FS':'sum',
        '+/-':'sum',
        'MIN':'sum'})

    tab_sum['FG%'] = round(tab_sum['FGM'] / tab_sum['FGA'] * 100, 2)
    tab_sum['FT%'] = round(tab_sum['FTM'] / tab_sum['FTA'] * 100, 2)
    tab_sum['2P%'] = round(tab_sum['2PM'] / tab_sum['2PA'] * 100, 2)
    tab_sum['3P%'] = round(tab_sum['3PM'] / tab_sum['3PA'] * 100, 2)

    # games per player
    games_per_player = tabs.groupby('PLAYER', as_index=False).size().rename(columns={'size':'G'})

    tab_mean = pd.merge(
        pd.merge(
            tab_mean,
            tab_sum[['PLAYER','FG%','FT%','2P%','3P%']],
            on='PLAYER',
            how='left'            
        ),
        games_per_player,
        on='PLAYER',
        how='left'
    )

    tab_sum = pd.merge(
        tab_sum,
        games_per_player,
        on='PLAYER',
        how='left'
    )

    tab_mean.MIN = tab_mean.MIN.apply(lambda x: fix_mins(x))
    tab_sum.MIN = tab_sum.MIN.apply(lambda x: fix_mins(x))

    # prepare output
    cols_order = ['PLAYER','Age','POS','HEIGHT','G',
                  'PTS','FGM','FGA','FG%','3PM','3PA','3P%','2PM','2PA','2P%','FTM','FTA','FT%',
                  'RO','RD','RT','AST','PR','PP','ST','FF','FS','+/-','MIN']

    team_cols_order = ['Game','PTS','FGM','FGA','FG%','3PM','3PA','3P%','2PM','2PA','2P%','FTM','FTA','FT%',
                       'RO','RD','RT','AST','PR','PP','ST','FF','FS']

    tab_sum = tab_sum[cols_order]
    tab_mean = tab_mean[cols_order]
    team_stats = team_stats[team_cols_order]

In [7]:
team_stats

,Game,PTS,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,FTM,FTA,FT%,RO,RD,RT,AST,PR,PP,ST,FF,FS
0,vs Rozzano,69,24,71,33.80,5,17,29.41,19,54,35.19,16,22,72.73,12,27,39,15,6,13,0,31,21
1,@ Tromello,69,28,57,49.12,5,7,71.43,23,50,46.00,8,13,61.54,10,28,38,18,7,13,2,12,13
2,vs Tumminelli,61,23,57,40.35,2,5,40.00,21,52,40.38,13,19,68.42,3,29,32,19,6,11,4,14,18
3,@ Trezzano,52,19,60,31.67,1,11,9.09,18,49,36.73,13,18,72.22,5,28,33,10,11,21,6,16,17
4,@ Aurora,53,22,66,33.33,1,17,5.88,21,49,42.86,8,16,50.00,11,27,38,12,9,14,1,22,20
5,vs Voghera,74,30,83,36.14,4,16,25.00,26,67,38.81,10,17,58.82,14,42,56,19,11,10,3,18,16
6,@ Garegnano,52,20,69,28.99,6,25,24.00,14,44,31.82,6,16,37.50,13,29,42,17,10,15,1,19,16
7,vs Milano3,74,28,73,38.36,8,23,34.78,20,50,40.00,10,26,38.46,15,29,44,15,10,15,0,14,21


In [8]:
opp_team_stats

,Game,FTM,FTA,2PM,2PA,3PM,3PA,PTS
0,vs Rozzano,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,@ Tromello,12.0,17.0,11.0,30.0,8.0,25.0,58.0
2,vs Tumminelli,9.0,17.0,19.0,40.0,4.0,13.0,59.0
3,@ Trezzano,10.0,12.0,22.0,48.0,5.0,27.0,69.0
4,@ Aurora,11.0,21.0,16.0,37.0,7.0,21.0,64.0
5,vs Voghera,19.0,23.0,17.0,54.0,4.0,23.0,65.0
6,@ Garegnano,11.0,14.0,20.0,46.0,6.0,25.0,69.0
7,vs Milano3,13.0,16.0,18.0,37.0,13.0,30.0,88.0
